# PyTorch, Autograd и все-все-все
Источники: 
- http://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html
- https://towardsdatascience.com/pytorch-autograd-understanding-the-heart-of-pytorchs-magic-2686cd94ec95

In [204]:
%matplotlib inline

In [205]:
%%bash
#conda install pytorch torchvision -c soumith

PyTorch Tensor/Autograd
============

Простыми словами Tensor - это n-мерный массив в PyTorch. Особенность тензоров Pytorch состоит в том, что для них эффективно реализованы различные тензорные операции, с возможностью исполнения как на CPU, так и на GPU. В совопкупности с модулем __torch.autograd__ , они организуют высокопроизводительный интерфейс для построения графов вычислений с возможностью вычисления градиентов по узлам графа. Данный граф носит название - динамического графа вычислений(DCG), листья графа - входные тензора, корни - выходные. Градиенты считаются по пути от корней к листьям, при помощи chain rule.

*Ранее был доп. класс torch.autorgrad.Variable для создания тензоров с поддержкой операций дифференцирования, но сейчас он deprecated*

Параметры объекта __torch.Tensor__ используемые autorgrad'ом:
 - __.requires_grad__ = [True,False] нужно ли рассчитываеть градиенты для данного тензора. Возможно управление данным параметром через метод __.detach()__ - отключает дальнейшее вычисление градиентов для данного тензора. Так же можно использовать "безградиентный контекст" __with torch.no_grad(): ...__ 
 - __.grad_fn__ - ссылка на функцию которая будет использована для вычисления градиентов
 - __.grad__ - значения градиента для данного тензора(считается на основе .grad_fn)
 - __.is_leaf__ - флаг, казывающий является ли данный тензор листом. 

In [1]:
import torch
import numpy as np

x = torch.randn(2, 2, requires_grad = True)
print(x)

# From numpy
x = np.array([1., 2., 3.]) #Only Tensors of floating point dtype can require gradients
x = torch.from_numpy(x)
# Now enable gradient 
x.requires_grad_(True)
print(x)
# _ above makes the change in-place (its a common pytorch thing)

tensor([[-0.6258,  0.3750],
        [-0.7126, -0.7372]], requires_grad=True)
tensor([1., 2., 3.], dtype=torch.float64, requires_grad=True)


In [207]:
print(x.data)

tensor([1., 2., 3.], dtype=torch.float64)


In [2]:
x = torch.ones(2,2, requires_grad=True)

In [3]:
print(x.grad)

None


In [4]:
print(x.grad_fn)  # we've created x ourselves

None


Проведем операции над x:



In [5]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


y был создан в результате операции сложения, поэтому у него должен быть grad_fn



In [6]:
print(y.grad_fn)

Еще поупражняемся с y:



In [7]:
z = y * y * 3
out = z.mean()

print(z)
print(out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward1>)


In [8]:
print("Y is leaf : {}".format(y.is_leaf))
print("Y req_grad : {}".format(y.requires_grad))

print("Z is leaf : {}".format(z.is_leaf))
print("Z req_grad : {}".format(z.requires_grad))

print("X is leaf : {}".format(x.is_leaf))
print("X req_grad : {}".format(x.requires_grad))

Y is leaf : False
Y req_grad : True
Z is leaf : False
Z req_grad : True
X is leaf : True
X req_grad : True


Градиенты
---------

Давайте прогоним backprop и получим d(out)/dx

In [9]:
out.backward()
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


Рассмотрим пример, когда в качестве корня графа мы получаем тензор

In [10]:
x = torch.ones(2, 2)
x.requires_grad_(True)
y = x + 2
y.backward(torch.ones(2, 2))
# the retain_variables flag will prevent the internal buffers from being freed
print(x.grad)

tensor([[1., 1.],
        [1., 1.]])


In [11]:
z = y * y
print(z)

tensor([[9., 9.],
        [9., 9.]], grad_fn=<MulBackward0>)


In [12]:
gradient = torch.randn(2, 2, requires_grad=False)

# this would fail if we didn't specify
# that we want to retain variables
for i in range(3):
    y.backward(gradient)
    print(x.grad)

tensor([[-0.4022,  0.4708],
        [ 0.1264,  1.7786]])
tensor([[-1.8045, -0.0584],
        [-0.7473,  2.5572]])
tensor([[-3.2067, -0.5875],
        [-1.6209,  3.3358]])


In [13]:
print("Y is leaf : {}".format(y.is_leaf))
print("Y req_grad : {}".format(y.requires_grad))

print("X is leaf : {}".format(x.is_leaf))
print("X req_grad : {}".format(x.requires_grad))

Y is leaf : False
Y req_grad : True
X is leaf : True
X req_grad : True
